<a href="https://colab.research.google.com/github/naenumtou/learnTensorFlow/blob/main/faceMaskDetection/faceMaskVideo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Set auto reload
%reload_ext autoreload
%autoreload 2

In [ ]:
# Change working directory
%cd /content/drive/My Drive/Colab Notebooks/face_mask_detection

/content/drive/My Drive/Colab Notebooks/face_mask_detection


In [ ]:
# Import libraries
import tensorflow as tf
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras.models import load_model

# Checking GPU available
if tf.test.gpu_device_name() == '/device:GPU:0':
  print('GPU is available')
else:
  print('GPU is not available')

print(f'Tensorflow version: {tf.__version__}')

# Config
%config InlineBackend.figure_format = 'retina' #Retina display

GPU is available
Tensorflow version: 2.7.0


In [ ]:
# Define parameters
labels = ['Masked', 'No mask']
size = 224
model = load_model('face_mask.model')

In [ ]:
# Utilise 'facenet' model for face detection
faceConfig = 'deploy.prototxt.txt'
faceModel = 'res10_300x300_ssd_iter_140000.caffemodel'

# Load 'facenet' model
faceNet = cv2.dnn.readNet(
    faceConfig,
    faceModel
)

In [ ]:
# Define function to predict with video file
def videoFaceMask(fileVideo):
  cap = cv2.VideoCapture(fileVideo)
  w, h  = int(cap.get(3)), int(cap.get(4))
  outVideo = cv2.VideoWriter(
      'videoResult' + os.path.splitext(fileVideo)[0][-1] + '.mp4',
      cv2.VideoWriter_fourcc(*'MP4V'), #With '.mp4' format
      30,
      (w, h)
  )
  while True:
    ret, frame = cap.read()
    if ret == True:
      (fh, fw) = frame.shape[:2]
      blob = cv2.dnn.blobFromImage(
          frame,
          1.0,
          (300, 300),
          (104.0, 177.0, 123.0)
      ) #Mean scale
      faceNet.setInput(blob)
      detections = faceNet.forward()
      for i in range(0, detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence < 0.5:
          continue
        box = detections[0, 0, i, 3:7] * np.array([fw, fh, fw, fh])
        (startX, startY, endX, endY) = box.astype('int')
        (startX, startY) = (max(0, startX), max(0, startY))
        (endX, endY) = (min(fw - 1, endX), min(fh - 1, endY))
        face = frame[startY:endY, startX:endX][:, :, ::-1]
        # Image processing
        face = cv2.resize(face, (size, size))
        face = np.reshape(face, (1, size, size, 3)) / 255.0
        # Model prediction
        pred = model.predict(face, batch_size = 64) #Same as training model
        label = labels[np.argmax(pred)] #Get label
        if label == labels[0]:
          color = (0, 255, 0)
        else:
          color = (0, 0, 255)
        cv2.rectangle(
            frame,
            (startX, startY),
            (endX, endY),
            color,
            3
        )
        cv2.rectangle(
            frame,
            (startX, startY - 60),
            (endX, startY),
            color,
            -1
        )
        cv2.putText(
            frame,
            label,
            (startX + 10, startY - 10),
            cv2.FONT_HERSHEY_SIMPLEX,
            1.5,
            (255, 255, 255),
            2
        )
        outVideo.write(frame)
    else:
      break
  return

In [ ]:
# Testing with video
for fileName in sorted(os.listdir(os.getcwd())):
  if fileName.endswith('.mp4'):
    print(f'Processing: {fileName}')
    videoFaceMask(fileName)

Processing: vidoeTest1.mp4
Processing: vidoeTest2.mp4
Processing: vidoeTest3.mp4
Processing: vidoeTest4.mp4
